In [1]:

import cv2 as cv
import math
import numpy as np
from matplotlib import pyplot as plt
import random as rng
import glob
from IPython.display import Image, display
import random 

sift = cv.SIFT_create()
bf1 = cv.BFMatcher()
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
orb = cv.ORB_create()
pad=1000
showall=False
showsome=True
scale=0.2
l3=True
l2=True
l4=True
folder='james'
flann = cv.FlannBasedMatcher()

In [2]:
imgData = {
  "kp": [],
  "d": [],
    'skp':[],
    'sd':[],
  "image": []
}

matchData = {
  "matches": [],
    'matrix':[],
  "score": [],
}


pairData = {
  "idx": 0,
  "score": 1000,
  "matchdata":matchData,
    "matrix1":[],
    "matrix2":[]
}

info = {
  "matches":[],
    "score":[],
    "matrix":[],
      "imgdata": [],
  "pairs": [],
    'total':0
}
inimg = []
inimg2 = []

In [3]:
def getKeyp(folder):
    d=[]
    kp=[]
    img=[]
    xa=[]
    xa1=[]
    a=0

    print('finding data...')
    il=[]
    for imageName in glob.glob(folder+'/*.jpg'):
        print(a)
        a=a+1
        i1 = cv.imread(imageName)
        i1=cv.resize(i1, (int(i1.shape[1]*scale),int(i1.shape[0]*scale)), interpolation = cv.INTER_AREA)
        i1=addPad(i1).copy()
#         kp1, d1 = orb.detectAndCompute(i1,None)#key points and detectors
        kp2, d2 = sift.detectAndCompute(i1,None)#key points and detectors
        imgData['kp']=kp2
        imgData['d']=d2
#         imgData['skp']=kp1
#         imgData['sd']=d1
#         imgData['skp']=kp2
#         imgData['sd']=d2
        imgData['image']=i1.copy()
        img.append(imgData.copy())
    for i in range(a):
        xa1.append(0)
        xa.append([])
        info['pairs'].append([])
    for i in range(a):
        info['matrix'].append(xa.copy())
        info['score'].append(xa1.copy())
        info['matches'].append(xa.copy())
    info['total']=a
    print('data found')
#     print(imgData)
#     print(info)
    info['imgdata']=img
    return info.copy()

In [4]:
def getData(d1,d2,k1,k2,i,j):
    s=0
    qu=[]
    tr=[]
    kp1=[]
    kp2=[]
    thresh=30
    good_points =[]
    matches=[]
    bmatches=[]
    matches = flann.knnMatch(d1,d2,k=2)
#     bmatches = bf.match(d11,d22)
#     print(matches)
#     orderedmatches = sorted(bmatches, key = lambda x:x.distance)
    th=0.25
    if folder == 'room' or folder == 'map':
        th=0.5
    for m,n in matches:
#         if 2*orderedmatches[0].distance>thresh/2:
        s=s+m.distance      
        if(m.distance < th*n.distance):
            good_points.append(m)

    info["matches"][i][j]=good_points.copy()
#     info["matches"][j][i]=good_points.copy()
    
    info["score"][i][j]=(s/len(matches))
    good_points=[]
    return (s/len(matches))

In [5]:
def getPairs(folder):
    info=getKeyp(folder)
#     print(info)
    data=[]
    pairs=[]
    pair=[]
    pair1=[]
    matchData=[]
    thresh=255
    if folder == 'office':
        thresh=290
    elif folder == 'test':
        thresh = 250
    elif folder == 'james':
        thresh=250
    elif folder == 'room':
        thresh=350
    elif folder == 'ac':
        thresh=270
    elif folder == 'map':
        thresh=295
    elif folder == 'turtle':
        thresh=295
    elif folder == 'WLH':
        thresh=265

    print('getting pairs...')            
    for i in range(0,info['total']-1):
        print(str(int((100*(i+1))/info['total']))+"%")
        for j in range(0,info['total']):
            if not i == j:
                score= getData(info['imgdata'][i]["d"],info['imgdata'][j]["d"],info['imgdata'][i]["kp"],info['imgdata'][j]["kp"],i,j)
#             print(info["score"])
                info["score"][i][j]=score
#             print(info["score"])
            #             print(matchData["score"])
#             office:20
#             james:25    

#             print(score)
                if(score<thresh)and len(info['matches'][i][j])>6:
#                 pair.append(j)
#                 pair1.append(i)
                    info['pairs'][i].append(j)
#                 info['pairs'][j].append(i)
        info['pairs'][i]= sorted(info['pairs'][i], key = lambda x:info['score'][i][x])
        pair=[]
        pair1=[]
    print('pairs found')
    return info.copy()

In [6]:
def  getMat():
    print("start mat finding...")
    mat = []
    tempmat = []
    M=[]
    for p in range(info["total"]-1):
        if len(info["pairs"][p]) ==0:
            print("***ERROR***")
            
        for i in range(len(info["pairs"][p])):
            xq=info["pairs"][p][i]
            p1 = np. float32([info['imgdata'][p]['kp'][m.queryIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)
            p2 = np. float32([info['imgdata'][info["pairs"][p][i]]['kp'][m.trainIdx].pt for m in info['matches'][p][info['pairs'][p][i]]]).reshape(-1,1,2)     
            info['matrix'][p][info["pairs"][p][i]] = cv.findHomography(p2, p1, cv.RANSAC,5.0)[0]
        
    return

In [7]:
def saveMe(p,i):
    print(p)
    print(i)
    p1 = np. float32([info['imgdata'][p]['kp'][m.queryIdx].pt for m in info['matches'][p][i]]).reshape(-1,1,2)
    p2 = np. float32([info['imgdata'][i]['kp'][m.trainIdx].pt for m in info['matches'][p][i]]).reshape(-1,1,2)
    M, mask = cv.findHomography(p2, p1, cv.RANSAC,5.0)
    print("Saved fewf...")
    return M

In [8]:
def getCenter():
    min = 0
    mini=0
    sum=0
    sums=[]
    avglen=0
    for i in range(info['total']-1):
        sum=0
        for p in info['score'][i]:
#             print(p)
            sum=sum+p
        tem = len(info['score'][i])
        avglen=avglen+len(info['pairs'][i])
        if tem ==0:
            tem=1
            sum=1000000
            
        sums.append(sum/tem)    
            
#         if len(info['pairs'][i])==0:
#             sums.append(1000)
#         else:
#             sums.append(sum/len(info['pairs'][i]))\
    avglen=avglen/info['total']
    print(avglen)
    for i in range(info['total']-1):
        if sums[i]<sums[mini] and len(info['pairs'][i])>= avglen:
            mini=i
    return mini

In [9]:
def addimg(i,m):
    if showall or showsome:
        print("add start")
        print("")
    gray = cv.cvtColor(m, cv.COLOR_BGR2GRAY)
    thresh = cv.threshold(gray, 2, 255, cv.THRESH_BINARY)[1]
    a,b= np.nonzero(thresh)
    for c in range(len(a)):
        i[a[c]][b[c]]=m[a[c]][b[c]]
    if showall or showsome:
        print("add done")
        print("")
#     plt.imshow(img1),plt.show()
    return i.copy()

In [10]:
def blur(image):
    blurred_img = cv.GaussianBlur(image, (201, 201), 0)
    mask = np.zeros(image.shape, np.uint8)
    mask1 = np.zeros(image.shape, np.uint8)

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     print(cv.threshold(gray, 60, 255, cv.THRESH_BINARY))
    thresh = cv.threshold(gray, 2, 255, cv.THRESH_BINARY)[1]
    contours, hierarchy = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    cv.drawContours(mask, contours, -1, (255,255,255),int(30*scale))
    output = np.where(mask==np.array([255, 255, 255]), mask1, image)
    return output.copy()

In [11]:
def newH(img1,img2):
    good_points=[]

    kp1, d1 = sift.detectAndCompute(img1,None)#key points and detectors
    kp2, d2 = sift.detectAndCompute(img2,None)#key points and detectors
    matches = flann.knnMatch(d1,d2,k=2)

    for m,n in matches:
        if(m.distance < 0.7*n.distance):
            good_points.append(m)
    p1 = np. float32([kp1[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
    p2 = np. float32([kp2[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
    H=cv.findHomography(p1, p2)[0]
    return H

In [12]:
def stitchimg(comp,i1,i2,flag,img):
    img1= comp.copy()
    img2=info['imgdata'][i2]['image'].copy()
    H=info['matrix'][i1][i2]
    if flag:
        img2=img.copy()
#         H=newH(img1,img2)
    
    if len(H) == 0:
        H=saveMe(i1,i2)
    if showall or showsome:
        print("stitching together:")
        print(i1)
        print(i2)
        print(" ")
    warped1 = cv.warpPerspective(blur(img2.copy()),H , (img2.shape[1],img2.shape[0]))
    if showall:
        plt.figure(figsize=(20,10))
        print('warped')
        plt.imshow(cv.cvtColor(warped1, cv.COLOR_BGR2RGB)),plt.show()    
    ret = addimg(img1.copy(),warped1).copy()
    
    return ret

In [13]:
def addPad(img):
    ret=img.copy()
    row, col = ret.shape[:2]
    bottom = ret[row-2:row, 0:col]
    mean = cv.mean(bottom)[0]

#     bordersize = int((ret.shape[0]+ret.shape[1])/4)
#     print(int((img.shape[0]+img.shape[1])))
    bordersize=int(pad*1.5)
    b1=bordersize
    bt=b1
    bb=b1
    b2=b1
    if folder=='office':
        b1=2*bordersize
        b2=b1
        bt=int(bordersize/2)
        bb=int(bordersize/2)
    if folder == 'james':
        bt=int(bt/4 )
        bb=bb+int((3*bb)/4)
        
    if folder == 'WLH':
        bt=int(bt/3 )
        bb=int(bb)
        b2=b1+100
        b1=b1+500
    border = cv.copyMakeBorder(ret,top=int(bt),bottom=int(bb),left=int(b2),right=int(b1 ),borderType=cv.BORDER_CONSTANT,value=[0, 0, 0])
    return border.copy()

In [14]:
def Convert_xy(x, y):
    global center, f

    xt = ( f * np.tan( (x - center[0]) / f ) ) + center[0]
    yt = ( (y - center[1]) / np.cos( (x - center[0]) / f ) ) + center[1]
    
    return xt, yt


def ProjectOntoCylinder(InitialImage):
    global w, h, center, f
    h, w = InitialImage.shape[:2]
    center = [w // 2, h // 2]
    f = 1100       # 1100 field; 1000 Sun; 1500 Rainier; 1050 Helens
    
    # Creating a blank transformed image
    TransformedImage = np.zeros(InitialImage.shape, dtype=np.uint8)
    
    # Storing all coordinates of the transformed image in 2 arrays (x and y coordinates)
    AllCoordinates_of_ti =  np.array([np.array([i, j]) for i in range(w) for j in range(h)])
    ti_x = AllCoordinates_of_ti[:, 0]
    ti_y = AllCoordinates_of_ti[:, 1]
    
    # Finding corresponding coordinates of the transformed image in the initial image
    ii_x, ii_y = Convert_xy(ti_x, ti_y)

    # Rounding off the coordinate values to get exact pixel values (top-left corner)
    ii_tl_x = ii_x.astype(int)
    ii_tl_y = ii_y.astype(int)

    # Finding transformed image points whose corresponding 
    # initial image points lies inside the initial image
    GoodIndices = (ii_tl_x >= 0) * (ii_tl_x <= (w-2)) * \
                  (ii_tl_y >= 0) * (ii_tl_y <= (h-2))

    # Removing all the outside points from everywhere
    ti_x = ti_x[GoodIndices]
    ti_y = ti_y[GoodIndices]
    
    ii_x = ii_x[GoodIndices]
    ii_y = ii_y[GoodIndices]

    ii_tl_x = ii_tl_x[GoodIndices]
    ii_tl_y = ii_tl_y[GoodIndices]

    # Bilinear interpolation
    dx = ii_x - ii_tl_x
    dy = ii_y - ii_tl_y

    weight_tl = (1.0 - dx) * (1.0 - dy)
    weight_tr = (dx)       * (1.0 - dy)
    weight_bl = (1.0 - dx) * (dy)
    weight_br = (dx)       * (dy)
    
    TransformedImage[ti_y, ti_x, :] = ( weight_tl[:, None] * InitialImage[ii_tl_y,     ii_tl_x,     :] ) + \
                                      ( weight_tr[:, None] * InitialImage[ii_tl_y,     ii_tl_x + 1, :] ) + \
                                      ( weight_bl[:, None] * InitialImage[ii_tl_y + 1, ii_tl_x,     :] ) + \
                                      ( weight_br[:, None] * InitialImage[ii_tl_y + 1, ii_tl_x + 1, :] )


    # Getting x coorinate to remove black region from right and left in the transformed image
    min_x = min(ti_x)

    # Cropping out the black region from both sides (using symmetricity)
    TransformedImage = TransformedImage[:, min_x : -min_x, :]

    return TransformedImage, ti_x-min_x, ti_y

In [15]:

def stitch():
    c=getCenter()
    print('center is:')
    print(c)
    print(info['pairs'][c])
    retImg = info['imgdata'][c]['image'].copy()
    inimg.append(c)
    te=0
    for i in info['pairs'][c]:
        inimg.append(i)
        temp1 = info['imgdata'][i]['image'].copy()
        print(str(int(te/len(info['pairs'][c])))+"%")
        
        for az in info['pairs'][i]:
#             if not l2:
#                 break
            temp12 = info['imgdata'][az]['image'].copy()
            if not az == c and  not az == i and l2 and not az in inimg:
                if (az in info['pairs'][c] and info['score'][i][az]>info['score'][c][az]) or not az in info['pairs'][c]:
                    if (az in info['pairs'][c] and info['score'][i][az]<info['score'][c][az]):
                        print('newStep**********')
#                         info['pairs'][c].remove(az)
                    inimg.append(az)
                   
                    for az1 in info['pairs'][az]:
#                     if not l3:
#                         break
                        if  not az1 in info['pairs'][c] and not az1 in inimg and not az1 in info['pairs'][i] and not az1 == c and not az1 == i and l3:
                            inimg.append(az1)
                            temp11 = info['imgdata'][az1]['image'].copy()
                            for az2 in info['pairs'][az1]:
                                temp111 = info['imgdata'][az2]['image'].copy()
                                if  not az2 in info['pairs'][c] and not az2 in inimg and not az2 in info['pairs'][i] and not az2 in info['pairs'][az] and not az2 == c and not az2 == i and not az2 == az and not az2 in inimg and l4:
                                    temp11=stitchimg(temp11.copy(),az1,az2,True,temp111).copy()
                                    inimg.append(az2)
                                    if showall:
                                        print('temp x3')
                                        plt.figure(figsize=(20,10))
                                        plt.imshow(cv.cvtColor(temp11, cv.COLOR_BGR2RGB)),plt.show()
                            temp12=stitchimg(temp12.copy(),az,az1,True,temp11).copy()
                            
                            if showall:
                                print('temp x2')
                                plt.figure(figsize=(20,10))
                                plt.imshow(cv.cvtColor(temp12, cv.COLOR_BGR2RGB)),plt.show()
                                
                    temp1=stitchimg(temp1.copy(),i,az,True,temp12.copy()).copy()
                if showall or showsome:
                    print('temp')
                    plt.figure(figsize=(20,10))
                    plt.imshow(cv.cvtColor(temp1, cv.COLOR_BGR2RGB)),plt.show()
        
        retImg=stitchimg(retImg,c,i,True,temp1.copy()).copy()
        te=te+100
        if showall or showsome:
            print('next step')
            plt.figure(figsize=(20,10))
            plt.imshow(cv.cvtColor(retImg, cv.COLOR_BGR2RGB)),plt.show()
    plt.figure(figsize=(20,10))
    plt.imshow(cv.cvtColor(retImg, cv.COLOR_BGR2RGB)),plt.show()
    plt.figure(figsize=(20,10))
    
    print("DONE!")
    return retImg
            

In [ ]:
%%time
l2=True
l3=True
l4=True
showall=False
showsome=False
folder='office'
pad =3000
scale=0.2

#************************************************************************
info = getPairs(folder)#*************************************************
#************************************************************************
print("start matrix")
getMat()
print(info['pairs'])
print("done matrix")
# print(info['score'])


finding data...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [ ]:
%%time
x=stitch()
print(inimg)

In [ ]:
# %%time
# inimg = []
# inimg2 = []
# info['matches']=[]
# info['score']=[]
# info['matrix']=[]
# info['imgdata']=[]
# info['pairs']=[]
# info['total']=0
# pad=3500
# scale=0.18
# l3=True
# l4=True
# showall=False
# folder='james'
# info = getPairs(folder)
# getMat()
# print(info['pairs'])


In [ ]:
# %%time
# x=stitch()
# print(inimg)

In [ ]:
%%time
inimg = []
inimg2 = []
info['matches']=[]
info['score']=[]
info['matrix']=[]
info['imgdata']=[]
info['pairs']=[]
info['total']=0
pad=3000
scale=0.15
l3=True
l4=True
showall=False
folder='office'
info = getPairs(folder)
getMat()
print(info['pairs'])

In [ ]:
%%time
x=stitch()
print(inimg)